# CNN Class
## An implementation of a Convolutional Neural Network using classes to classify the mnist handwritten digits database using the quickstart for experts guide on TensorFlow.

### Setup

In [14]:
# Import statements
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

TensorFlow version: 2.9.1


In [15]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train/255, x_test/255

# Add a channels dimension (Marked for review)
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# Batching and shuffling the dataset (Marked for review)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### Building the Model

In [16]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(64, (3,3), activation="relu")
        self.maxPooling1 = MaxPooling2D((2,2))
        self.flatten = Flatten()
        self.d1 = Dense(64, activation="relu")
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.maxPooling1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [17]:
# Choosing and optimizer and loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# This is the metrics for loss and accuracy
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")


In [18]:
# Training the Model

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training = True only needed for dropout and other like layers
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


In [19]:
# Test the model
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [20]:
EPOCHS = 5
for epoch in range(EPOCHS):
    # Reset metrics at the start of each epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels, in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )
    

Epoch 1, Loss: 0.3951050043106079, Accuracy: 85.99333190917969, Test Loss: 0.3035428822040558, Test Accuracy: 88.99000549316406
Epoch 2, Loss: 0.2709139585494995, Accuracy: 90.21499633789062, Test Loss: 0.2772497236728668, Test Accuracy: 89.92000579833984
Epoch 3, Loss: 0.2278342843055725, Accuracy: 91.70833587646484, Test Loss: 0.26262935996055603, Test Accuracy: 90.47000122070312
Epoch 4, Loss: 0.19647714495658875, Accuracy: 92.75833129882812, Test Loss: 0.2595815360546112, Test Accuracy: 90.69999694824219
Epoch 5, Loss: 0.16946983337402344, Accuracy: 93.7750015258789, Test Loss: 0.25823327898979187, Test Accuracy: 90.8699951171875


In [21]:
model.summary()


Model: "my_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           multiple                  640       
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         multiple                  0         
                                                                 
 dense_6 (Dense)             multiple                  692288    
                                                                 
 dense_7 (Dense)             multiple                  650       
                                                                 
Total params: 693,578
Trainable params: 693,578
Non-trainable params: 0
__________________________________________________